In [5]:
from  selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by  import By
from bs4 import BeautifulSoup
import time
import pandas as pd
from pymongo import MongoClient
import os


In [22]:
def extraction():
    driver = webdriver.Chrome()

    driver.implicitly_wait(10)
    driver.set_script_timeout(180)
    driver.set_page_load_timeout(10)
    driver.get('https://www.yelp.com/')
    time.sleep(5)


    text_input = driver.find_element(By.CSS_SELECTOR, "input[id='search_description']")
    text_input.send_keys('Indian \n')
    # Click the element
    time.sleep(5)
    base_directory = '/Users/himanishprakash/Desktop/DDR/Yelp_data'
    number_of_page = 10 
    for i in range(number_of_page):
        
        page_source = driver.page_source
        page_file_path = os.path.join(base_directory, f'yelp_indian_{i+1}.html')
        
        with open(page_file_path, 'w', encoding='utf-8') as f:
            f.write(page_source)
        
        ##  Clicking on all time Steals leader      
        page_1 = driver.find_element(By.CSS_SELECTOR, "button[class='pagination-button__09f24__kbFYf css-x61rbl']")
        page_1.click()
        time.sleep(6)
        
        

    driver.quit()
    
    

In [25]:
def transformation_load():
    # Specify the directory containing your HTML files
    directory_path = '/Users/himanishprakash/Desktop/DDR/Yelp_data'

    # List all files in the directory
    html_files = [f for f in os.listdir(directory_path) if f.endswith('.html')]

    # Sort the files by name to maintain order, if necessary
    html_files.sort()
    extracted_values = []

    # Loop through each HTML file
    for html_file in html_files:
        # Construct the full path to the HTML file
        file_path = os.path.join(directory_path, html_file)
        
        # Open and read the HTML file
        with open(file_path, 'r', encoding='utf-8') as file:
            html_content = file.read()
        
        # Process the HTML content, e.g., using BeautifulSoup to parse it
        soup = BeautifulSoup(html_content, 'html.parser')
        
        # Find all elements matching the given CSS selector
        Restaurant_name = soup.select('a.css-19v1rkv')
        location = soup.select('span.css-chan6m')
        for name, loc in zip(Restaurant_name, location):
            extracted_values.append({'Restaurant Name': name.text.strip(), 'Location': loc.text.strip()})

    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(extracted_values)

    # Drop duplicates based on both 'Restaurant Name' and 'Location'
    df_unique = df.drop_duplicates(subset=['Restaurant Name', 'Location'])

    df_unique = df_unique.iloc[3:]
    df_unique = df_unique[~df_unique['Location'].str.contains('reviews')]
    df_unique = df_unique[~df_unique['Location'].str.contains('review')]
    # Define the path for the CSV file to save the unique values
    csv_file_path = '/Users/himanishprakash/Desktop/DDR/Yelp_data/extracted_values.csv'

    # Save the DataFrame with unique values to a CSV file
    df_unique.to_csv(csv_file_path, index=False)

    print(f"Unique restaurant names and locations saved to {csv_file_path}")
        

In [26]:
extraction()
transformation_load()

Unique restaurant names and locations saved to /Users/himanishprakash/Desktop/DDR/Yelp_data/extracted_values.csv
